<a href="https://colab.research.google.com/github/Rutuja-Kolte/CodeBrewers/blob/master/CodeBrewers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Setting Up**

In [1]:
# To import everything needed for the project

%matplotlib inline
from google.colab import files
import os

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [ ]:
# To mount drive and access files from it

from google.colab import drive
drive.mount('/content/drive')

In [3]:
# Used to create a symbolic link (symlink)

!ln -s "/content/drive/My Drive/CodeBrewers" "/content/CodeBrewers"

In [ ]:
# To unzip files (images) from proj.zip

!unzip /content/CodeBrewers/proj.zip

**Loading Images**

In [ ]:
# To get all the paths for the images to later load them
imagepaths = []
# To go through all the files and subdirectories inside the folder and save the path to the list
for root, dirs, files in os.walk(".", topdown=False): 
  for name in files:
    path = os.path.join(root, name)
    if path.endswith("png"):  # To append only images
      imagepaths.append(path)
print(len(imagepaths))

In [6]:
# Plots image for debugging
def plot_image(path):
  img = cv2.imread(path)  # Reads the image into a numpy array
  img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # To convert the original image from the BGR color space to gray
  print(img_cvt.shape)  # Prints the shape of the image
  plt.grid(False) # To plot image without grid
  plt.imshow(img_cvt) # Shows the image
  plt.xlabel("Width")
  plt.ylabel("Height")
  plt.title("Image " + path)

In [ ]:
plot_image(imagepaths[0]) # Plots the first image from the imagepaths array

In [ ]:
# To load all the images into an array, X, and all the labels into an array, Y

X = []
Y = []
for path in imagepaths:
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (320, 120)) # Reduces image size
  X.append(img)
  category = path.split("/")[3] # To get label from path
  label = int(category.split("_")[0][1])
  Y.append(label)
# To convert X and Y into numpy arrays
X = np.array(X, dtype="uint8")
X = X.reshape(len(imagepaths), 120, 320, 1)
Y = np.array(Y)
print("Images loaded: ", len(X))
print("Labels loaded: ", len(Y))
print(Y[0], imagepaths[0])  # Prints label and path of 1st image for debugging

**Creating Model**

In [ ]:
# To import keras model and hidden layers for the Convolutional Neural Network (CNN)
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten

In [10]:
# To construct the model

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(120, 320, 1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [11]:
# To configure the model for training

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# To train the model

model.fit(X, Y, epochs=5, batch_size=64, verbose=2, validation_data=(X, Y))

In [ ]:
# To save the model as an HDF5 file

model.save('GestureRecognition.h5')